# EHR Project Extract/Transform/Load

In [1]:
# from __future__ import absolute_import, division, print_function, unicode_literals
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_probability as tfp
# import tensorflow_data_validation as tfdv # blursed library
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import aequitas as ae

In [2]:
# Put all of the helper functions in utils
from utils import build_vocab_files, show_group_stats_viz, aggregate_dataset, preprocess_df, df_to_dataset, posterior_mean_field, prior_trainable
pd.set_option('display.max_columns', 500)

# this allows you to make changes and save in student_utils.py and the file is reloaded every time you run a code block
%load_ext autoreload
%autoreload

In [3]:
#OPEN ISSUE ON MAC OSX for TF model training
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [4]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
dataset_path = "./data/final_project_dataset.csv"
df = pd.read_csv(dataset_path)

## Reduce Dimensionality of the NDC Code Feature

**Question 3**: NDC codes are a common format to represent the wide variety of drugs that are prescribed for patient care in the United States. The challenge is that there are many codes that map to the same or similar drug. You are provided with the ndc drug lookup file https://github.com/udacity/nd320-c1-emr-data-starter/blob/master/project/data_schema_references/ndc_lookup_table.csv derived from the National Drug Codes List site(https://ndclist.com/). Please use this file to come up with a way to reduce the dimensionality of this field and create a new field in the dataset called "generic_drug_name" in the output dataframe. 

In [6]:
#NDC code lookup file
ndc_code_path = "./medication_lookup_tables/final_ndc_lookup_table"
ndc_code_df = pd.read_csv(ndc_code_path)

In [7]:
from student_utils import reduce_dimension_ndc

In [8]:
reduce_dim_df = reduce_dimension_ndc(df, ndc_code_df)

In [9]:
# Number of unique values should be less for the new output field
assert df['ndc_code'].nunique() > reduce_dim_df['generic_drug_name'].nunique()

In [10]:
reduce_dim_df.generic_drug_name.nunique() / df.ndc_code.nunique()

0.09163346613545817

In [11]:
reduce_dim_df.generic_drug_name.nunique()

23

In [12]:
reduce_dim_df.generic_drug_name.unique()

array(['None', 'Human Insulin', 'Glipizide', 'Insulin Human',
       'Metformin Hydrochloride', 'Glimepiride', 'Glyburide',
       'Rosiglitazone Maleate', 'Repaglinide', 'Acarbose', 'Tolbutamide',
       'Tolazamide', 'Pioglitazone', 'Metformin Hcl',
       'Pioglitazone Hydrochloride And Glimepiride',
       'Glyburide-metformin Hydrochloride', 'Nateglinide',
       'Glyburide And Metformin Hydrochloride', 'Miglitol',
       'Glipizide And Metformin Hydrochloride',
       'Glipizide And Metformin Hcl', 'Pioglitazone And Glimepiride',
       'Pioglitazole And Metformin Hydrochloride'], dtype=object)

## Select First Encounter for each Patient 

**Question 4**: In order to simplify the aggregation of data for the model, we will only select the first encounter for each patient in the dataset. This is to reduce the risk of data leakage of future patient encounters and to reduce complexity of the data transformation and modeling steps. We will assume that sorting in numerical order on the encounter_id provides the time horizon for determining which encounters come before and after another.

In [13]:
from student_utils import select_first_encounter
first_encounter_df = select_first_encounter(reduce_dim_df)

In [14]:
# unique patients in transformed dataset
unique_patients = first_encounter_df['patient_nbr'].nunique()
print("Number of unique patients:{}".format(unique_patients))

# unique encounters in transformed dataset
unique_encounters = first_encounter_df['encounter_id'].nunique()
print("Number of unique encounters:{}".format(unique_encounters))

original_unique_patient_number = reduce_dim_df['patient_nbr'].nunique()
# number of unique patients should be equal to the number of unique encounters and patients in the final dataset
assert original_unique_patient_number == unique_patients
assert original_unique_patient_number == unique_encounters
print("Tests passed!!")

Number of unique patients:71518
Number of unique encounters:71518
Tests passed!!


## Aggregate at the first-encounter level

We need to ensure that every encounter is only a single row, and since we have already changed to one encounter per patient this further means that the number of patients, rows, and encounters should all be equal.

In [15]:
agg = first_encounter_df.groupby('encounter_id').agg(lambda c: [x for x in c if x is not None]).reset_index()

In [16]:
def aggfn(x):
    if type(x) == list:
        if len(np.unique(x)) == 1:
            return x[0]
        else:
            return x
    else:
        return x

agg = agg.applymap(aggfn)

In [17]:
agg.generic_drug_name = agg.generic_drug_name.map(lambda x: [x] if type(x) is str else x)

In [18]:
assert agg.patient_nbr.nunique() == agg.encounter_id.nunique() == len(agg)

## Feature Selection and Dummy Vars

Our dataframe has changed a lot in terms of the rows dimension but has changed very little in terms of columns so the work we did above in the EDA can carry over to here.

Previously we identified usable `numeric` columns, usable `categorical` columns and a final group `other` which includes UIDs and columns that are not worth inclusion in the final dataset. Note also that while UIDs are essential for splitting purposes we need to ensure they are removed before training as they can cause overfitting and other issues. Another note is that, compared to the EDA, the `admission_X_id` fields have been removed here as they didn't seem to have any noticeable correlation with our dependent variable.

The one outlier here is `generic_drug_name` in which the patient can have multiple categories in one field, so we will need to deal with that on its own. 

In [19]:
numeric = [
    'number_outpatient', 
    'number_inpatient', 
    'number_emergency', 
    'num_lab_procedures',
    'number_diagnoses',
    'num_medications',
    'num_procedures']

categorical = [
    'race',
    'gender',
    'age',
    'max_glu_serum',
    'A1Cresult',
    'change',
    'readmitted'
]

ids = [
    'encounter_id',
    'patient_nbr'
]

target = ['time_in_hospital']

other = [
    'admission_type_id',
    'discharge_disposition_id',
    'admission_source_id',
    'other_diagnosis_codes',
    'primary_diagnosis_code',
    'ndc_code',
    'medical_specialty',
    'payer_code',
    'weight',
    'generic_drug_name'
]

In [20]:
print(f"""
Numeric: {len(numeric)}
Categorical: {len(categorical)}
ID/Other: {len(other) + len(ids) + len(target)}
Total: {len(numeric) + len(categorical) + len(other) + len(ids) + len(target)}
""")


Numeric: 7
Categorical: 7
ID/Other: 13
Total: 27



In [21]:
len(agg.columns)

27

In [22]:
agg[numeric].dtypes

number_outpatient     int64
number_inpatient      int64
number_emergency      int64
num_lab_procedures    int64
number_diagnoses      int64
num_medications       int64
num_procedures        int64
dtype: object

In [23]:
agg[categorical].dtypes

race             object
gender           object
age              object
max_glu_serum    object
A1Cresult        object
change           object
readmitted       object
dtype: object

### Creating Dummy Vars

Most of these can be handled automatically but we need to pay special attention to `generic_drug_name` which contains multiple values at once.

Here we handle `generic_drug_name` using an approach that can identify multiple dummy variables in the same data field. Each of the first three examples includes multiple medications and the final example (index 4) contains only a single medication.

In [24]:
generic_drug_dummies = agg.generic_drug_name.str.join(sep=',').str.get_dummies(sep=',')

In [25]:
generic_colnames = generic_drug_dummies.columns.str.replace(' ', '_').str.lower()
generic_drug_dummies.columns = generic_colnames

In [26]:
generic_drug_dummies.head(3)

,acarbose,glimepiride,glipizide,glipizide_and_metformin_hcl,glipizide_and_metformin_hydrochloride,glyburide,glyburide_and_metformin_hydrochloride,glyburide-metformin_hydrochloride,human_insulin,insulin_human,metformin_hcl,metformin_hydrochloride,miglitol,nateglinide,none,pioglitazole_and_metformin_hydrochloride,pioglitazone,pioglitazone_hydrochloride_and_glimepiride,repaglinide,rosiglitazone_maleate,tolazamide,tolbutamide
0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
2,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


### Normalizing Numeric Values

Not normalizing the dependent variable here.

In [41]:
normcols = agg[numeric].apply(lambda c: (c - np.mean(c)) / (np.std(c) + 1e-8))
numcols = pd.concat([normcols, agg[target].astype(float)], axis=1)

In [42]:
numcols.head(3)

,number_outpatient,number_inpatient,number_emergency,num_lab_procedures,number_diagnoses,num_medications,num_procedures,time_in_hospital
0,-0.262004,-0.294524,-0.203346,1.249212,0.378159,1.479343,0.323563,13.0
1,-0.262004,-0.294524,-0.203346,-0.504981,0.378159,0.276134,0.891793,12.0
2,-0.262004,-0.294524,-0.203346,0.397175,-1.125856,-0.927076,-0.812896,1.0


### Adding a Bias Columns

In [43]:
bias = pd.DataFrame({'bias': np.ones(len(agg))})

### Concatenating the Dataset

Now we select our columns from the aggregated dataset, concat the drug dummy vars and use the automatic dummy generation to get the rest.

In [44]:
data = pd.concat([agg[ids], bias, numcols, agg[categorical], generic_drug_dummies], axis=1)

In [45]:
len(data.columns)

40

**Question 5**: After you have aggregated the dataset to the right level, we can do feature selection (we will include the ndc_col_list, dummy column features too). In the block below, please select the categorical and numerical features that you will use for the model, so that we can create a dataset subset. 

For the payer_code and weight fields, please provide whether you think we should include/exclude the field in our model and give a justification/rationale for this based off of the statistics of the data. Feel free to use visualizations or summary statistics to support your choice.

In [46]:
df.payer_code.str.startswith('?').astype(int).sum() / len(df)

0.37783076751450245

In [47]:
df.weight.str.startswith('?').astype(int).sum() / len(df)

0.9700050200803213

_Student response:_ As mentioned in the EDA both `payer_code` and `weight` have 'hidden' missing values in the form of the `'?'` string which represents missing without using zero or NaN. In both cases a large number of values are missing, and the missing category is in fact the most common value. With `weight` 97% of the values are missing meaning the field contains almost no information. As for `payer_code` the field seems primarily billing related, so in addition to the missing information it's questionable as to whether the remaining information is even of clinical use vis-a-vis our dependent variable.

## Prepare Fields and Cast Dataset 

In [48]:
'''
Please update the list to include the features you think are appropriate for the model 
and the field that we will be using to train the model. There are three required demographic features for the model 
and I have inserted a list with them already in the categorical list. 
These will be required for later steps when analyzing data splits and model biases.
'''
required_demo_col_list = ['race', 'gender', 'age']
student_categorical_col_list = [c for c in data.columns if c not in ids + numeric]
student_numerical_col_list = numeric
PREDICTOR_FIELD = 'time_in_hospital'

In [49]:
selected_features_df = data

### Preprocess Dataset - Casting and Imputing  

We will cast and impute the dataset before splitting so that we do not have to repeat these steps across the splits in the next step. For imputing, there can be deeper analysis into which features to impute and how to impute but for the sake of time, we are taking a general strategy of imputing zero for only numerical features. 

OPTIONAL: What are some potential issues with this approach? Can you recommend a better way and also implement it?

### Response

Via feature selection and EDA we ended up with no columns that had missing or NaN values aside from the `'?'` string. As mentioned in the EDA the fields that still contain `'?'` are useful enough even with missing values and in this case given that we have no good statistical basis for imputing other values we will keep those as-is.

```python
processed_df = preprocess_df(selected_features_df, student_categorical_col_list, 
        student_numerical_col_list, PREDICTOR_FIELD, categorical_impute_value='nan', numerical_impute_value=0)
```

In [50]:
processed_df = data

In [51]:
processed_df.to_csv('processed_data.csv')

In [52]:
processed_df.head(10)

,encounter_id,patient_nbr,bias,number_outpatient,number_inpatient,number_emergency,num_lab_procedures,number_diagnoses,num_medications,num_procedures,time_in_hospital,race,gender,age,max_glu_serum,A1Cresult,change,readmitted,acarbose,glimepiride,glipizide,glipizide_and_metformin_hcl,glipizide_and_metformin_hydrochloride,glyburide,glyburide_and_metformin_hydrochloride,glyburide-metformin_hydrochloride,human_insulin,insulin_human,metformin_hcl,metformin_hydrochloride,miglitol,nateglinide,none,pioglitazole_and_metformin_hydrochloride,pioglitazone,pioglitazone_hydrochloride_and_glimepiride,repaglinide,rosiglitazone_maleate,tolazamide,tolbutamide
0,12522,48330783,1.0,-0.262004,-0.294524,-0.203346,1.249212,0.378159,1.479343,0.323563,13.0,Caucasian,Female,[80-90),None,None,Ch,NO,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,15738,63555939,1.0,-0.262004,-0.294524,-0.203346,-0.504981,0.378159,0.276134,0.891793,12.0,Caucasian,Female,[90-100),None,None,Ch,NO,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
2,16680,42519267,1.0,-0.262004,-0.294524,-0.203346,0.397175,-1.125856,-0.927076,-0.812896,1.0,Caucasian,Male,[40-50),None,None,Ch,NO,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,28236,89869032,1.0,-0.262004,-0.294524,-0.203346,0.196696,0.879498,0.155813,0.323563,9.0,AfricanAmerican,Female,[40-50),None,None,No,>30,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,35754,82637451,1.0,-0.262004,-0.294524,-0.203346,-0.605220,0.879498,0.035492,2.596482,3.0,Caucasian,Male,[50-60),None,None,No,>30,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
5,36900,77391171,1.0,-0.262004,-0.294524,-0.203346,0.948493,-0.123179,-0.566113,-0.812896,7.0,AfricanAmerican,Male,[60-70),None,None,Ch,<30,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
6,40926,85504905,1.0,-0.262004,-0.294524,1.760584,0.848254,0.378159,-0.084829,-0.812896,7.0,Caucasian,Female,[40-50),None,None,Ch,<30,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0
7,42570,77586282,1.0,-0.262004,-0.294524,-0.203346,0.597655,0.378159,1.840306,-0.244667,10.0,Caucasian,Male,[80-90),None,None,No,NO,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
8,55842,84259809,1.0,-0.262004,-0.294524,-0.203346,1.349451,-0.123179,0.637096,-0.244667,4.0,Caucasian,Male,[60-70),None,None,Ch,NO,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
9,62256,49726791,1.0,-0.262004,-0.294524,-0.203346,0.296936,0.378159,-1.649002,2.028252,1.0,AfricanAmerican,Female,[60-70),None,None,No,>30,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
